In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
!pip install holidays
import holidays
%matplotlib inline

    100% |████████████████████████████████| 71kB 505kB/s 
  Running setup.py bdist_wheel for holidays ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/6f/d2/1a/5854fb5211d237db046d9ae62674a112d020d84c40a729dd02
Successfully built holidays
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
def get_distributions(salesMonth):
    salesMonth.aggVODKA = salesMonth.aggVODKA / salesMonth.numRecords
    salesMonth.aggRUM = salesMonth.aggRUM / salesMonth.numRecords
    salesMonth.aggBRANDIES = salesMonth.aggBRANDIES / salesMonth.numRecords
    salesMonth.aggTEQUILA = salesMonth.aggTEQUILA / salesMonth.numRecords
    salesMonth.aggSCHNAPPS = salesMonth.aggSCHNAPPS / salesMonth.numRecords
    salesMonth.aggWHISKIES = salesMonth.aggWHISKIES / salesMonth.numRecords
    salesMonth.aggSCOTCH = salesMonth.aggSCOTCH / salesMonth.numRecords
    salesMonth.aggOTHER = salesMonth.aggOTHER / salesMonth.numRecords
    salesMonth.aggLIQUEURS = salesMonth.aggLIQUEURS / salesMonth.numRecords
    return salesMonth
def clean_cities(sales):
    spellingMapping = {
    "clearlake": "clear lake",
    "dewitt": "de witt",
    "guttenburg" : "guttenberg",
    "grand mounds": "grand mound",
    "jewell": "jewell junction",
    "kellog": "kellogg",
    "ottuwma": "ottumwa",
    "otumwa": "ottumwa",
    "lemars": "le mars",
    "leclaire": "le claire"
    }   
    sales['City'] = sales['City'].map(lambda x: x.lower().replace("'", "").strip())
    sales['City'] = sales['City'].map(lambda x: spellingMapping.get(x, x))
    sales['City'] = sales['City'].map(lambda x: x.replace('mt', 'mount'))
    sales['City'] = sales['City'].map(lambda x: x.replace('saint', 'st.'))
    sales['City'] = sales['City'].map(lambda x: re.sub('^ft.', 'fort', x))
    sales['City'] = sales['City'].map(lambda x: re.sub('^st ', 'st. ', x))
    return sales

In [32]:
#Aggregate by City and Day
sales = pd.read_csv('GrossSalesCountsDay.csv')
sales.Date = pd.to_datetime(sales.Date, infer_datetime_format=True)
sales = clean_cities(sales)
sales = get_distributions(sales)

census = pd.read_csv("CensusData.csv")
census['City'] = census['City'].map(lambda x: x.lower().replace("'", "").strip())
censusunique = np.unique(census['City'])
for city in np.unique(sales['City']):
    if city not in censusunique:
        print(city)
        
merged = sales.merge(census, on='City', how='inner')
merged = merged.drop("Unnamed: 0", axis = 1)
merged['Volume Per Capita'] = merged['Volume Sold (Liters)'] / merged['Population']
merged['Bottles Per Capita'] = merged['Bottles Sold'] / merged['Population']
merged['Sales Per Capita'] = merged['Sale (Dollars)'] / merged['Population']

one_hot_city = pd.get_dummies(merged.City)
merged = pd.merge(merged, one_hot_city, left_index=True, right_index=True)


merged.Date = pd.to_datetime(merged.Date, infer_datetime_format=True)
# The day of the week with Monday=0, Sunday=6
merged['Dayofweek'] = merged.Date.dt.dayofweek
# The week ordinal of the year
merged['Week'] = merged.Date.dt.weekofyear
merged['Month'] = merged.Date.dt.month

#print("merged.Date = {} and type = {}".format(merged.Date, type(merged.Date)))
us_holidays = holidays.UnitedStates()
merged['Holiday'] = merged.apply(lambda x: x['Date'] in us_holidays, axis =1)
merged.to_csv("GrossSalesDay1.csv", index = False)

colorado springs


In [4]:
#aggregate by City, Year, and Month
sales = pd.read_csv('GrossSalesCountsDay.csv')
sales.Date = pd.to_datetime(sales.Date, infer_datetime_format=True)
sales = clean_cities(sales)

sales['Month'] = sales.Date.dt.month
sales['Year'] = sales.Date.dt.year
us_holidays = holidays.UnitedStates()
sales['Holiday Sales'] = sales.apply(lambda x: x['Date'] in us_holidays, axis =1).astype(int)

salesMonth = sales.groupby(['City', 'Year', 'Month']).sum().reset_index()
sales = get_distributions(salesMonth)

census = pd.read_csv("CensusData.csv")
census['City'] = census['City'].map(lambda x: x.lower().replace("'", "").strip())
censusunique = np.unique(census['City'])
for city in np.unique(sales['City']):
    if city not in censusunique:
        print(city)
merged = sales.merge(census, on='City', how='inner')
merged = merged.drop("Unnamed: 0", axis = 1)
merged['Volume Per Capita'] = merged['Volume Sold (Liters)'] / merged['Population']
merged['Bottles Per Capita'] = merged['Bottles Sold'] / merged['Population']
merged['Sales Per Capita'] = merged['Sale (Dollars)'] / merged['Population']

one_hot_city = pd.get_dummies(merged.City)
merged = pd.merge(merged, one_hot_city, left_index=True, right_index=True)

merged.to_csv("GrossSalesCityYearMonth.csv" , index=False)
month_data = merged

colorado springs


In [36]:
#Aggregate by City, Year, and Week
sales = pd.read_csv('GrossSalesCountsDay.csv')
sales.Date = pd.to_datetime(sales.Date, infer_datetime_format=True)
sales = clean_cities(sales)

sales['Week'] = sales.Date.dt.weekofyear
sales['Year'] = sales.Date.dt.year

us_holidays = holidays.UnitedStates()
sales['Holiday Sales'] = sales.apply(lambda x: x['Date'] in us_holidays, axis =1).astype(int)
salesMonth = sales.groupby(['City', 'Year', 'Week']).sum().reset_index()
sales = get_distributions(salesMonth)
census = pd.read_csv("CensusData.csv")
census['City'] = census['City'].map(lambda x: x.lower().replace("'", "").strip())
censusunique = np.unique(census['City'])
for city in np.unique(sales['City']):
    if city not in censusunique:
        print(city)
merged = sales.merge(census, on='City', how='inner')
merged = merged.drop("Unnamed: 0", axis = 1)
merged['Volume Per Capita'] = merged['Volume Sold (Liters)'] / merged['Population']
merged['Bottles Per Capita'] = merged['Bottles Sold'] / merged['Population']
merged['Sales Per Capita'] = merged['Sale (Dollars)'] / merged['Population']
one_hot_city = pd.get_dummies(merged.City)
merged = pd.merge(merged, one_hot_city, left_index=True, right_index=True)

merged.to_csv("GrossSalesCityYearWeek.csv", index=False)


colorado springs
